In [1]:
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [16]:
# PARAMETROS
VALIDATION_SPLIT = 0.15

# RUTA DE DATASET MS
DATASET_MS = '../dataset/ms/dataset-ms.xlsx'

In [3]:
# CARGA DE DATOS HOJA TRAIN
train_ds = pd.read_excel(DATASET_MS, sheet_name='train')

In [4]:
# DIVISION DE DATOS DE ENTRADA (X) Y SALIDA (y)
X = train_ds.iloc[:, :-1] 
y = train_ds.iloc[:, -1]

In [13]:
#VISUALIZAMOS LOS DATOS DE ENTRADA
X.head()

,ResistenciaEsperada,Cemento,Agua,AgregadoFino,AgregadoGrueso,TiempoCurado
0,350,662,238,742,745,28
1,280,493,239,681,1025,1
2,210,407,239,855,858,1
3,280,493,239,817,820,3
4,280,443,217,869,872,7


In [14]:
#VISUALIZAMOS LOS DATOS DE SALIDA
y.head()

0    29150.0
1     4600.0
2     3940.0
3    11230.0
4    13840.0
Name: Carga, dtype: float64

In [17]:
# DIVIDIMOS EL DATASET EN ENTRENAMIENTO Y VALIDACION
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=VALIDATION_SPLIT, random_state=42 , stratify=X['ResistenciaEsperada'])

In [19]:
print('Cantidad de datos de entrenamiento: ', len(X_train))
print('Cantidad de datos de validación: ', len(X_val))

Cantidad de datos de entrenamiento:  286
Cantidad de datos de validación:  51


In [ ]:
# Normalizar tus características de entrada
""" scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test) """
# Normalizar tus características de entrada
X_train = (X_train - X_train.mean()) / X_train.std()
X_val = (X_val - X_train.mean()) / X_train.std()
X_test = (X_test - X_train.mean()) / X_train.std()

y_train = (y_train - y_train.mean()) / y_train.std()
y_val = (y_val - y_train.mean()) / y_train.std()
y_test = (y_test - y_train.mean()) / y_train.std()

In [ ]:
# Crear el modelo de DNN
model = Sequential()
model.add(Dense(64, input_dim=5, activation='relu'))  # 5 características de entrada
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))  # Salida lineal para la resistencia del cemento


In [ ]:
# Compilar el modelo
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])  # Cambiamos 'accuracy' por 'mae'


In [ ]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

In [ ]:
import matplotlib.pyplot as plt

# Recoge los valores de pérdida y accuracy de la historia
loss = history.history['loss']
val_loss = history.history['val_loss']
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Crea una gráfica para la pérdida
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Crea una gráfica para el accuracy
plt.subplot(1, 2, 2)
plt.plot(accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Evaluar el modelo en el conjunto de prueba
loss = model.evaluate(X_test, y_test)
print(f'Pérdida en el conjunto de prueba: {loss}')

In [ ]:
# Hacer predicciones
predictions = model.predict(X_test)